In [1]:
import time

import pandas as pd
from shapely.geometry.collection import GeometryCollection
from shapely.geometry.multipolygon import MultiPolygon

from app.gen_planner.python.src import GenPlanner, default_terr_zones, default_func_zones
import geopandas as gpd
from app.gen_planner.python.src.utils import (
    explode_linestring,
    extend_linestring,
    geometry_to_multilinestring,
    territory_splitter,
)

roads = gpd.read_file('svetogorsk/roads.geojson')[['geometry']]
existing_zones = gpd.read_file('svetogorsk/exist_terr_zones.geojson')
territory = gpd.read_file('svetogorsk/orig_bounds.geojson')
water = gpd.read_file('svetogorsk/water.geojson')
# rails = gpd.read_file('svetogorsk/rails.geojson')
# rails.geometry = rails.to_crs(32636).buffer(8)

# exclude_features = pd.concat([water.to_crs(4326), rails.to_crs(4326)], ignore_index=True)

terr_zone_map = {'Жилая зона': default_terr_zones.residential_terr,
                 'Рекреационная зона': default_terr_zones.recreation_terr,
                 'Промышленная зона': default_terr_zones.industrial_terr,
                 'Особого назначения': default_terr_zones.special_terr,
                 }
existing_zones['terr_zone'] = existing_zones['Тип функциональной зоны'].map(terr_zone_map)

gp = GenPlanner(features=territory, roads=roads, exclude_features=water,
                simplify_value=20,
                parallel=True)


2025-10-13 18:49:35.660 | WARNING  | app.gen_planner.python.src.genplanner.genplanner:_create_working_gdf:129 - Column 'roads_width' missing in GeoDataFrame, filling it with default local road width 5


In [2]:
gp = GenPlanner(features=territory,
                exclude_features=water,
                simplify_value=20,
                parallel=True)


In [3]:
gdf1,gdf2 = gp.features2terr_zones2blocks(default_func_zones.residential_func_zone)

In [5]:
gp.territory_to_work_with.explore()

In [4]:
m1 =  gdf1.to_crs(32635).explore(column='territory_zone', tiles='cartodb positron')
gdf2.explore(m=m1)

In [ ]:
m1 = pd.concat([gp.existing_terr_zones.to_crs(32635).rename(columns={'terr_zone': 'territory_zone'}), gdf1.to_crs(32635)],
          ignore_index=True).explore(column='territory_zone', tiles='cartodb positron')
gdf2.explore(m=m1)

In [ ]:
roads.explore()

In [ ]:
m1 = gp.existing_terr_zones.explore(column='terr_zone', tiles='cartodb positron')
gdf1.explore(m=m1, column='territory_zone_name')

In [ ]:

gdf = terr.copy()
local_crs = gdf.crs
existing_terr_zones = existing_zones[['terr_zone', 'geometry']].copy()

existing_terr_zones = existing_terr_zones.to_crs(local_crs)
existing_terr_zones = existing_terr_zones.clip(gdf)

# self.existing_terr_zones = existing_terr_zones
splitted_territory = territory_splitter(gdf, existing_terr_zones, return_splitters=True).reset_index(drop=True)

splitted_territory['existing_area'] = splitted_territory.area
splitted_territory.geometry = splitted_territory.representative_point()
splitted_territory = splitted_territory.sjoin(existing_terr_zones, how='left').rename(
    columns={'index_right': 'existing_zone_index'})
splitted_territory = splitted_territory[~splitted_territory['terr_zone'].isna()]

gdf['full_area'] = gdf.area
potential = gdf.sjoin(splitted_territory, how='left')
potential = potential[~potential['terr_zone'].isna()]
potential['ratio'] = (potential['existing_area'] / potential['full_area']).round(2)
consistent_idx = (
    potential.groupby(level=0)["terr_zone"]
    .nunique()
    .pipe(lambda s: s[s == 1].index)
)
potential = potential.loc[consistent_idx]
potential: gpd.GeoDataFrame = potential[(potential['ratio'] >= 0.8)]

for ezi, group in potential.groupby('existing_zone_index'):
    base = existing_terr_zones.at[ezi, "geometry"]
    merged = unary_union([base, *group.geometry.dropna().to_list()])
    existing_terr_zones.at[ezi, "geometry"] = merged





In [ ]:
m1 = existing_terr_zones.explore(color='red')
existing_zones.explore(m=m1)
m1

In [ ]:
from shapely.ops import nearest_points, unary_union
import numpy as np

existing_zones_repr = existing_zones.to_crs(32635).copy()
existing_zones_repr.geometry = existing_zones_repr.geometry.representative_point()

target = terr.geometry.buffer(-0.5, resolution=1)
target_boundary = target.boundary.union_all()

projected = existing_zones_repr.copy()
projected.geometry = projected.geometry.apply(
    lambda p: nearest_points(p, target_boundary)[1]
)

m1 = existing_zones.explore(color='red')
terr.explore(m=m1)
projected.explore(m=m1, color='purple')
existing_zones_repr.explore(m=m1, color='pink')


In [ ]:
gpd.GeoDataFrame(geometry=[target_boundary], crs=32635).explore()

In [ ]:
projected

In [ ]:
from app.gen_planner.python.src.utils import (
    explode_linestring,
    extend_linestring,
    geometry_to_multilinestring,
    territory_splitter,
)

res = territory_splitter(terr, existing_zones, True)
res.explore()

In [ ]:
res = territory_splitter(territory, existing_zones, True)

In [ ]:
from app.gen_planner.python.src.zoning import func_zones, terr_zones

from app.gen_planner.python.src import GenPlanner
import geopandas as gpd

roads = gpd.read_file('svetogorsk/roads.geojson')
existing_zones = gpd.read_file('svetogorsk/exist_terr_zones.geojson')
territory = gpd.read_file('svetogorsk/orig_bounds.geojson')
water = gpd.read_file('svetogorsk/water.geojson')

terr_zone_map = {'Жилая зона': terr_zones.residential_terr,
                 'Рекреационная зона': terr_zones.recreation_terr,
                 'Промышленная зона': terr_zones.industrial_terr,
                 'Особого назначения': terr_zones.special_terr,
                 }
existing_zones['terr_zone'] = existing_zones['Тип функциональной зоны'].map(terr_zone_map)

gp = GenPlanner(features=territory, roads=roads, exclude_features=water, existing_terr_zones=existing_zones)

terr = gp.territory_to_work_with


In [ ]:
terr.explore()

In [ ]:

gdf1, gdf2 = gp.features2terr_zones2blocks(func_zones.residential_func_zone)

In [ ]:
gdf1['territory_zone_name'] = gdf1['territory_zone'].apply(lambda x: x.name)

In [ ]:
gdf1.explore(column='territory_zone_name')